## Business Objective

The goal of this analysis is to understand the drivers of Fulfillment Efficiency across
products, customers, and logistics operations, and to identify actionable levers that
can reduce returns, cancellations, and revenue loss.


## Defining Fulfillment Efficiency

Fulfillment Efficiency measures how much of the gross order value is ultimately realized
as delivered revenue after accounting for returns.


Fulfillment Efficiency = (Delivered Revenue − Returned Revenue) / Gross Order Value


## Key Metrics Used

- Gross Order Value (GOV)
- Delivered Revenue
- Returned Revenue
- Cancellation Rate
- Return Rate
- Fulfillment Efficiency


## Hypotheses

H1: Categories with lower product ratings have higher return rates.

H2: Products with low stock availability experience higher cancellation rates.

H3: Price deviations of ±10% from category averages are associated with poorer
fulfillment outcomes.

H4: Certain logistics combinations (shipping + payment method) drive higher returns.


In [ ]:
import gdown
import pandas as pd

## Data Import

The following cells download raw datasets from cloud storage.
These are setup steps and not part of the analysis.


In [ ]:
# @title
file_id = "1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "orders.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK
From (redirected): https://drive.google.com/uc?id=1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK&confirm=t&uuid=9d35a083-0db3-4f2b-a84f-d3baa3e6a811
To: /content/orders.csv
100%|██████████| 377M/377M [00:04<00:00, 79.0MB/s]


'orders.csv'

In [ ]:
file_id = "1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "orderline.csv", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8
From (redirected): https://drive.google.com/uc?id=1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8&confirm=t&uuid=d21e714b-7136-44f0-b773-d6cd8ea2eb9e
To: /content/orderline.csv
100%|██████████| 642M/642M [00:07<00:00, 82.2MB/s]


'orderline.csv'

In [ ]:
file_id = "1Aa5oSSE-3Fn6RQpupqcg2sAf3l2VlccA"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "person.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1Aa5oSSE-3Fn6RQpupqcg2sAf3l2VlccA
To: /content/person.csv
100%|██████████| 83.3M/83.3M [00:01<00:00, 68.4MB/s]


'person.csv'

In [ ]:
file_id = "1dL388NuXzV8mpTJ44HEmp2LgxKR1d4z8"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "product.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1dL388NuXzV8mpTJ44HEmp2LgxKR1d4z8
To: /content/product.csv
100%|██████████| 1.75M/1.75M [00:00<00:00, 17.3MB/s]


'product.csv'

In [ ]:
df_person = pd.read_csv("person.csv", sep=";")
df_orders = pd.read_csv("orders.csv", sep=";")
df_orderline = pd.read_csv("orderline.csv", sep=";")
df_product = pd.read_csv("product.csv", sep=";")

In [ ]:
df_orders.shape
df_orderline.shape
df_product.shape
df_person.shape

(600000, 12)

At this stage, no data cleaning was required as IDs and categorical fields were already well structured.

## Initial Data Inspection

To get a quick sense of the structure and fields in each table, I reviewed a small sample of rows from all datasets.  
This helps confirm column meanings and spot any obvious problems in data early.

In [ ]:
df_person['person_id'].is_unique
df_orders['order_id'].is_unique
df_product['product_id'].is_unique

True

In [ ]:
df_orders['status'].value_counts()
df_orderline['status'].value_counts()

,count
status,
fulfilled,2602356
cancelled,2600481
returned,2599406
pending,2599167
shipped,2598590


## Defining Overall Fulfillment Efficiency

To assess fulfillment performance at a platform level, we define **Fulfillment Efficiency** as the proportion of total order value that is successfully realized after accounting for returns.

### Metric Definitions

- **Gross Order Value (GOV):**  
  The total value of all order line subtotals across the platform, regardless of fulfillment outcome.

- **Delivered Revenue:**  
  The sum of subtotals for order lines with a *fulfilled* status, representing successfully completed deliveries.

- **Returned Revenue:**
  The sum of subtotals for order lines that were *returned*, representing revenue loss after delivery.

- **Net Realized Revenue (NRR):**  
  The actual revenue retained by the platform after subtracting returned revenue from delivered revenue.

  
  NRR = Delivered Revenue - Returned Revenue
  

- **Fulfillment Efficiency:**
  A normalized metric that measures how effectively gross order value is converted into retained revenue.

  Fulfillment Efficiency = NRR\GOV
  

This metric establishes a **baseline benchmark** for overall fulfillment performance and serves as a reference point for further analysis across products, categories, brands, and logistics operations.

In [ ]:
success_lines = ['fulfilled']
loss_lines = ['returned']

delivered_revenue = df_orderline.loc[
    df_orderline['status'].isin(success_lines), 'subtotal'
].sum()

returned_revenue = df_orderline.loc[
    df_orderline['status'].isin(loss_lines), 'subtotal'
].sum()

GOV = df_orderline['subtotal'].sum()

NRR = delivered_revenue - returned_revenue
fulfillment_efficiency = NRR / GOV
NRR, fulfillment_efficiency


(np.float64(10433400.990003586), np.float64(0.00014825130582453696))

This represents the overall fulfillment efficiency across the platform.


In [ ]:
df_combined = (
    df_orderline
        .merge(
            df_product[
                [
                    'product_id',
                    'category',
                    'brand',
                    'rating_average',
                    'review_count',
                    'price',
                    'stock_quantity'
                ]
            ],
            on='product_id',
            how='left'
        )
)

In [ ]:
category_summary = df_combined.groupby('category').agg(
    GOV=('subtotal','sum'),
    Delivered=('subtotal', lambda x: x[df_combined.loc[x.index,'status']=='fulfilled'].sum()),
    Returned=('subtotal', lambda x: x[df_combined.loc[x.index,'status']=='returned'].sum())
)

category_summary['Fulfillment_Efficiency'] = (
    (category_summary['Delivered'] - category_summary['Returned']) /
    category_summary['GOV']
)

category_summary.sort_values('Fulfillment_Efficiency')


,GOV,Delivered,Returned,Fulfillment_Efficiency
category,,,,
Beverages,1.472792e+09,2.925245e+08,2.977884e+08,-0.003574
Gaming,1.364663e+09,2.708127e+08,2.747989e+08,-0.002921
Kitchenware,1.174737e+09,2.326127e+08,2.356252e+08,-0.002564
Automotive,1.237309e+09,2.449236e+08,2.480962e+08,-0.002564
Reference,1.164652e+09,2.314727e+08,2.341926e+08,-0.002335
Car Electronics,1.259669e+09,2.518688e+08,2.545248e+08,-0.002108
Health,1.371456e+09,2.738700e+08,2.761382e+08,-0.001654
Audiobooks,1.510470e+09,3.007802e+08,3.029629e+08,-0.001445
Computing,1.534296e+09,3.069052e+08,3.088030e+08,-0.001237


#Fulfillment efficiency table
Several categories above show negative fulfillment efficiency due to return volumes exceeding fulfilled revenue. This indicates operational or product-level issues such as high return rates, customer dissatisfaction, or fragile logistics in those categories.

#Hypothesis: Lower product ratings lead to higher return rates

In [ ]:
df_combined.groupby(pd.cut(df_combined['rating_average'], bins=5))['status'] \
           .value_counts(normalize=True)


/tmp/ipython-input-408004261.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined.groupby(pd.cut(df_combined['rating_average'], bins=5))['status'] \


rating_average  status   
(0.996, 1.8]    shipped      0.200196
                fulfilled    0.200077
                cancelled    0.199992
                pending      0.199898
                returned     0.199838
(1.8, 2.6]      returned     0.200352
                fulfilled    0.200138
                cancelled    0.199917
                pending      0.199875
                shipped      0.199718
(2.6, 3.4]      fulfilled    0.200456
                cancelled    0.200208
                pending      0.199804
                shipped      0.199804
                returned     0.199729
(3.4, 4.2]      cancelled    0.200279
                returned     0.200074
                shipped      0.199902
                pending      0.199899
                fulfilled    0.199845
(4.2, 5.0]      fulfilled    0.200419
                pending      0.200222
                shipped      0.199800
                cancelled    0.199782
                returned     0.199777
Name: proportion, dtype: float64

### Impact of Product Ratings on Fulfillment Outcomes

Across all rating bands, order status distributions remain nearly uniform.
This suggests that product ratings alone are not a strong predictor of
returns, cancellations, or fulfillment success in this dataset.

As a result, further analysis focuses on operational and pricing factors
that may have a stronger influence on fulfillment efficiency.

No meaningful relationship is observed between product rating and fulfillment status. Status distributions remain nearly uniform across all rating bands.

#Hypothesis: Low stock availability increases cancellations



In [ ]:
df_combined.groupby(pd.cut(df_combined['stock_quantity'], bins=5))['status'] \
           .value_counts(normalize=True)

/tmp/ipython-input-3651899574.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined.groupby(pd.cut(df_combined['stock_quantity'], bins=5))['status'] \


stock_quantity  status   
(-0.5, 100.0]   cancelled    0.200117
                returned     0.200098
                fulfilled    0.200061
                pending      0.199921
                shipped      0.199803
(100.0, 200.0]  shipped      0.200247
                returned     0.199950
                fulfilled    0.199939
                pending      0.199937
                cancelled    0.199928
(200.0, 300.0]  fulfilled    0.200754
                returned     0.200002
                shipped      0.199840
                cancelled    0.199729
                pending      0.199676
(300.0, 400.0]  cancelled    0.200277
                fulfilled    0.200164
                returned     0.199891
                shipped      0.199850
                pending      0.199819
(400.0, 500.0]  pending      0.200342
                cancelled    0.200153
                fulfilled    0.199974
                returned     0.199823
                shipped      0.199707
Name: proportion, dtype: float64

### Interpretation

Across stock quantity buckets, the proportion of cancelled, returned, and fulfilled orders remains relatively stable.

This suggests that **stock availability alone does not strongly explain cancellation behavior** in this dataset.
Other factors such as logistics delays, customer behavior, or product-level quality issues are likely stronger drivers of cancellations.

#Hypothesis: ±10% price deviation leads to poorer fulfillment



In [ ]:
df_combined['price_band'] = pd.qcut(df_combined['price'], q=5)

df_combined.groupby('price_band')['status'] \
           .value_counts(normalize=True)

/tmp/ipython-input-251314353.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined.groupby('price_band')['status'] \


price_band          status   
(5.949, 392.99]     fulfilled    0.200448
                    shipped      0.200231
                    cancelled    0.199885
                    returned     0.199813
                    pending      0.199623
(392.99, 780.99]    cancelled    0.200104
                    fulfilled    0.200070
                    shipped      0.200065
                    returned     0.199890
                    pending      0.199871
(780.99, 1166.99]   returned     0.200340
                    cancelled    0.200208
                    pending      0.199966
                    fulfilled    0.199853
                    shipped      0.199634
(1166.99, 1580.99]  pending      0.200218
                    fulfilled    0.200116
                    cancelled    0.199991
                    shipped      0.199895
                    returned     0.199780
(1580.99, 1999.99]  fulfilled    0.200420
                    pending      0.200001
                    cancelled    0.199997
                    returned     0.199949
                    shipped      0.199632
Name: proportion, dtype: float64

#Conclusion:

Fulfillment outcomes remain stable across all price bands, with no observable increase in cancellations or returns for products priced significantly above or below category averages.

This suggests that pricing variation alone does not influence fulfillment efficiency in the current dataset, likely due to controlled or simulated pricing distributions.

## Hypothesis: Shipping Method Impacts Fulfillment Risk

### Hypothesis
Certain shipping methods are associated with higher fulfillment failure rates due to operational complexity, delivery speed constraints, and last-mile execution risk.

### Approach
- Define a fulfillment failure as any order with status:
  - Cancelled
  - Returned
  - Refunded
- Aggregate total orders and failures by shipping method
- Compute failure rate = failures / total orders

### Why this matters
Shipping methods directly impact customer experience and operational load. Identifying high-risk logistics options enables Amazon to optimize carrier selection, SLA commitments, and cost–reliability tradeoffs.

In [ ]:
shipping_risk = (
    df_orders
    .assign(is_failure=df_orders['status'].isin(['Cancelled', 'Returned', 'Refunded']))
    .groupby('shipping_method')
    .agg(
        total_orders=('order_id', 'count'),
        failures=('is_failure', 'sum')
    )
)

shipping_risk['failure_rate'] = shipping_risk['failures'] / shipping_risk['total_orders']
shipping_risk.sort_values('failure_rate', ascending=False)

,total_orders,failures,failure_rate
shipping_method,,,
overnight,624399,268137,0.429432
fedex,624949,268296,0.429309
express,625190,268324,0.429188
standard,624989,268231,0.429177
economy,625545,268466,0.429171
pickup_point,625081,268048,0.428821
dhl,624251,267544,0.428584
ups,625596,267575,0.427712


#Key Observations
Failure rates (Cancelled + Returned + Refunded orders) are consistently high across all shipping methods, ranging narrowly between ~42.8% and ~42.9%.

Premium delivery options such as overnight and express do not show lower failure rates compared to standard or economy shipping.

Third-party carriers (FedEx, DHL, UPS) exhibit only marginal differences, indicating no single shipping partner is disproportionately driving failures.

The minimal variance across shipping methods suggests that shipping choice is not a primary driver of fulfillment failures.



#Business Interpretation
Paying for faster shipping does not translate into better fulfillment outcomes for customers.

Fulfillment failures are likely driven upstream (product quality, inaccurate listings, customer expectations) rather than downstream logistics execution.

Optimizing shipping speed alone will have limited impact on reducing cancellations and returns.

## Hypothesis: Certain Brand–Category Combinations Drive Disproportionately High Return Rates

### Hypothesis
Specific brands within certain product categories contribute disproportionately to return volume, indicating potential issues related to product quality, sizing accuracy, or expectation mismatch.

### Methodology
- Joined order-line data with product catalog metadata
- Identified returned items at the line level
- Aggregated performance by category and brand
- Filtered out low-volume combinations (total lines ≤ 500) to ensure statistical significance
- Ranked brand–category pairs by return rate

### Output
The table below highlights the top brand–category combinations with the highest return rates.

In [ ]:
brand_category_risk = (
    df_combined
    .assign(is_returned=df_combined['status'] == 'returned')
    .groupby(['category', 'brand'])
    .agg(
        total_lines=('status', 'count'),
        returns=('is_returned', 'sum')
    )
)

brand_category_risk['return_rate'] = (
    brand_category_risk['returns'] / brand_category_risk['total_lines']
)

brand_category_risk\
    .query('total_lines > 500') \
    .sort_values('return_rate', ascending=False) \
    .head(10)

total_lines  returns  return_rate
category           brand                                          
Personal Care      ShowerStyle          1652      392     0.237288
Sports             GreenGrow            1651      391     0.236826
Childrens Clothing SpaceMaster          1661      393     0.236604
Formal Wear        OdeurOrange          1597      376     0.235441
Haircare           VitaminVault         1642      385     0.234470
Clothing           FanFresh             1604      375     0.233791
                   SuperFit             1662      387     0.232852
Sports             TrendyWear           1595      369     0.231348
Audiobooks         PillowPro            1652      382     0.231235
Wearables          StyleHub             1637      378     0.230910

### Key Observations
- High return rates are concentrated in apparel, personal care, and wearable-related categories.
- Returns are brand-specific rather than category-wide, suggesting catalog-level issues instead of fulfillment or logistics failures.
- These brands likely suffer from:
  - Inaccurate product descriptions
  - Sizing or fit inconsistencies
  - Quality perception gaps
  - Misleading imagery or marketing claims

### Business Implications
- High-return brands increase reverse logistics costs and inventory handling overhead.
- Persistent return behavior negatively impacts customer trust and repeat purchase likelihood.
- Blanket category-level interventions would be inefficient; targeted brand-level actions are required.

### Recommended Actions
1. Introduce stricter catalog quality audits for high-risk brands.
2. Enhance product detail pages with clearer sizing guides, usage instructions, and real customer imagery.
3. Apply return-rate-based penalties or incentives at the brand level.
4. Route high-return brands through slower but more cost-efficient fulfillment paths to reduce reverse logistics impact.
5. Flag these brand–category combinations for proactive customer expectation management.